In [1]:
!pip install tensorflow

     |████████████████████▏           | 302.3MB 157.3MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 22.9MB 101.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 96.0MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 95.8MB/s eta 0:00:01
     |████████████████████████████████| 133kB 103.7MB/s eta 0:00:01
     |████████████████████████████████| 5.3MB 94.6MB/s eta 0:00:01
     |████████████████████████████████| 317kB 110.8MB/s eta 0:00:01
     |████████████████████████████████| 5.6MB 98.4MB/s eta 0:00:01
     |████████████████████████████████| 450kB 90.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 91.6MB/s eta 0:00:01
     |████████████████████████████████| 1.7MB 77.2MB/s eta 0:00:01
     |████████████████████████████████| 2.4MB 86.9MB/s eta 0:00:01
     |████████████████████████████████| 4.8MB 90.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 87.5MB/s eta 0:00:01
     |████████████████████████████████| 808kB 100.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 90.1MB/s eta 0

     |████████████████████████████████| 133kB 109.3MB/s eta 0:00:01
     |████████████████████████████████| 184kB 106.2MB/s eta 0:00:01
     |████████████████████████████████| 153kB 107.9MB/s eta 0:00:01
     |████████████████████████████████| 92kB 110.3MB/s eta 0:00:01
ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement urllib3==1.26.15, but you'll have urllib3 2.0.7 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; python_version >= "3.8", but you'll have scikit-learn 1.3.0 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.


In [ ]:
input=tf.k

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [3]:
def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model


In [4]:
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


In [5]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential([generator, discriminator])
    return model


In [6]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [7]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


In [8]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [9]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [10]:
EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])


In [11]:
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)


In [16]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [15]:
for epoch in range(EPOCHS):
    for image_batch in dataset:
        train_step(image_batch)


NameError: name 'dataset' is not defined

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0  # Rescale from [-1, 1] to [0, 1]

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0], cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()


In [ ]:
for epoch in range(EPOCHS):
    generate_and_save_images(generator, epoch, seed)
